In [1]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    # df = pd.DataFrame(res_rows)
    field_names = [i[0] for i in cursor.description]
    return res_rows,field_names

INFO:root:connection created


In [2]:
from datetime import datetime

def nft_balance_cost_query(start_blo,end_blo):
    sql = """
INSERT INTO dm.dm_nft_balance_cost_eth_di
SELECT token_address
    ,token_id
    ,account_address
    ,`value` as balance_value
    ,block_number
    ,transaction_hash
    ,txn_block
    ,cost_eth_value
    ,cost_ts
    ,trade_type
    ,is_scalp
    ,now() as etl_time
FROM
(
    SELECT a.token_address
        ,a.token_id
        ,a.account_address
        ,a.`value`
        ,a.block_number
        ,b.cost_eth_value
        ,b.trade_type
        ,b.transaction_hash
        ,b.block_number as txn_block
        ,b.ts as cost_ts
        ,b.is_scalp
        ,row_number() over(partition by a.token_address,a.token_id ORDER BY b.ts DESC) as rnk
    FROM 
    (
        SELECT a.token_address,a.token_id,a.account_address,a.`value`,a.block_number,FROM_UNIXTIME(b.`timestamp`) as block_ts
        FROM dw.dws_nft_balance_eth as a
        left join prod.blocks as b
        ON a.block_number = b.block_number
        AND b.`chain` = 'eth'
        INNER JOIN dw.dwm_nft_detail_ha as c
        ON a.token_address = c.token_address
        AND c.token_type = 'ERC721'
        WHERE a.`value` > 0
        AND a.`block_number` >= {start_blo}
        AND a.`block_number` < {end_blo}
    ) as a
    LEFT JOIN 
    (
    select to_address as account_address
        ,token_address
        ,token_ids as token_id
        ,token_num
        ,buyer_pay_eth_amount as cost_eth_value
        ,trade_time as ts
        ,transaction_hash
        ,block_number
        ,trade_type
        ,is_scalp
    from dw.dwb_nft_trade_eth_detail_hi
    where erc_standard not in ('ERC1155')
    union all
    select account_address
        ,token_address
        ,token_id
        ,mint_amount as token_num
        ,(gas_cost+mint_cost) as cost_eth_value
        ,mint_time as ts
        ,transaction_hash
        ,block_number
        ,'mint' as trade_type
        ,0 as is_scalp
    from dw.dwb_nft_mint_detail_eth_hi
    ) as b
    ON a.token_address = b.token_address
    AND a.token_id = b.token_id
) as t
WHERE t.rnk = 1
""".format(
    start_blo=start_blo,
    end_blo=end_blo
)
    return sql

In [3]:
# 等分数据回溯区间
bucket_sql = """
SELECT block_number
    ,records
    ,cum_records
    ,rnk
FROM
(
    SELECT block_number
        ,records
        ,cum_records
        ,rank() over(PARTITION BY floor(cum_records/1000000) ORDER BY cum_records%1000000 asc) as rnk
    FROM
    (
        SELECT block_number
            ,records
            ,sum(records) over(ORDER BY block_number asc) as cum_records
        FROM
        (
            select block_number
                ,count(*) as records
            from dw.dws_nft_balance_eth
            where `value` > 0
            GROUP BY 1
        ) as a
    ) as aa
) as t
WHERE rnk = 1
"""
sql_res,sql_fields = getSqlResult(bucket_sql)

In [6]:
block_bucket = []
for row in sql_res:
    block_bucket.append(row[0])
    
print(len(block_bucket),sql_res[-1])
print(block_bucket)

122 (16616293, 169, 121000072, 1)
[952982, 7461281, 8176572, 9084750, 9598984, 11215892, 12172016, 12662317, 12927665, 13045371, 13130025, 13184843, 13264191, 13330526, 13397881, 13481451, 13617599, 13721504, 13808057, 13890958, 13953613, 14006867, 14049571, 14101199, 14144476, 14187784, 14225386, 14263410, 14295116, 14326837, 14358525, 14385179, 14419163, 14448687, 14481935, 14524401, 14555517, 14581487, 14602734, 14626201, 14645786, 14666086, 14696472, 14721824, 14740671, 14777434, 14796358, 14815526, 14828095, 14842704, 14857221, 14868163, 14879091, 14892471, 14906406, 14920483, 14934007, 14951684, 14979427, 15001927, 15027021, 15047485, 15066999, 15084045, 15104518, 15121524, 15141114, 15160033, 15184255, 15206426, 15232691, 15260275, 15282296, 15301952, 15324864, 15344747, 15361964, 15380020, 15398081, 15417582, 15434862, 15456635, 15475239, 15484219, 15506417, 15526547, 15544905, 15558418, 15573012, 15589383, 15607275, 15627566, 15648777, 15667247, 15688315, 15718550, 15756144, 1

In [13]:
from datetime import datetime
from datetime import timedelta

def backfill_data(block_bucket,start_blo=0,end_blo=0):
    start_blo = start_blo if start_blo > 0 else block_bucket[0]
    end_blo = end_blo if end_blo>0 else block_bucket[-1]
    for i in range(len(block_bucket)):
        current_blo = block_bucket[i]
        if current_blo < start_blo:
            continue
        elif current_blo >= end_blo:
            break
        else:
            current_end_blo = block_bucket[i+1]
            print(current_blo,current_end_blo)
            sql = nft_balance_cost_query(current_blo,current_end_blo)
            cursor.execute(sql)

backfill_data(block_bucket,start_blo=0,end_blo=14975000)

952982 7461281
7461281 8176572
8176572 9084750
9084750 9598984
9598984 11215892
11215892 12172016
12172016 12662317
12662317 12927665
12927665 13045371
13045371 13130025
13130025 13184843
13184843 13264191
13264191 13330526
13330526 13397881
13397881 13481451
13481451 13617599
13617599 13721504
13721504 13808057
13808057 13890958
13890958 13953613
13953613 14006867
14006867 14049571
14049571 14101199
14101199 14144476
14144476 14187784
14187784 14225386
14225386 14263410
14263410 14295116
14295116 14326837
14326837 14358525
14358525 14385179
14385179 14419163
14419163 14448687
14448687 14481935
14481935 14524401
14524401 14555517
14555517 14581487
14581487 14602734
14602734 14626201
14626201 14645786
14645786 14666086
14666086 14696472
14696472 14721824
14721824 14740671
14740671 14777434
14777434 14796358
14796358 14815526
14815526 14828095
14828095 14842704
14842704 14857221
14857221 14868163
14868163 14879091
14879091 14892471
14892471 14906406
14906406 14920483
14920483 14934007
14

1
2
3
